## Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import eurostat
import os

In [ ]:
from functools import reduce
from eurostat import get_data_df

## Research Questions

### 1. 

### 2.

### 3.

## Analysis